<a href="https://colab.research.google.com/github/Aadhil21/DSGP-Group28/blob/branch-Aadhil/Extract_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Install Libraries

In [54]:
!pip install mediapipe

#Import Libraries

In [55]:
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import numpy as np
import os
import time
import threading
import openpyxl
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision

#Initialize mediapipe model

In [65]:
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
pose = mp_pose.Pose(min_detection_confidence=0.2,
                    min_tracking_confidence=0.5,
                    static_image_mode=False,
                    smooth_landmarks=True,
                    model_complexity=1)

##Set Directory Path

In [74]:
if not os.path.exists('Videos'):
  os.mkdir('Videos')
if not os.path.exists('Datasets'):
  os.mkdir('Datasets')

video_path = '/content/DSGP-Group28/Videos/PushUp.mp4'
dataset_path = '/content/DSGP-Group28/Datasets/'

In [75]:
frame_count = 0
frame_numbers = []
data_dict = {}
desired_width = 480
desired_height = 360

In [76]:
cap = cv2.VideoCapture(video_path)

##Populate keypoints dictionary

In [77]:
for landmark in mp_pose.PoseLandmark:
  x_list_name = f"{landmark.name}_Point_x"
  y_list_name = f"{landmark.name}_Point_y"
  z_list_name = f"{landmark.name}_Point_z"
  data_dict[x_list_name] = []
  data_dict[y_list_name] = []
  data_dict[z_list_name] = []

##Process Frames

In [78]:
while cap.isOpened():
  success, image = cap.read()
  if not success:
    break

  # Increment frame_count
  frame_count += 1
  print(f'Processing frame {frame_count}')

  # Resize the frame to the desired width and height
  image = cv2.resize(image, (desired_width, desired_height))

  # Convert the image from BGR to RGB
  image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

  results = pose.process(image_rgb)

  if results.pose_landmarks:

    frame_numbers.append(frame_count)

    for landmark in mp_pose.PoseLandmark:
      x_list_name = f"{landmark.name}_Point_x"
      y_list_name = f"{landmark.name}_Point_y"
      z_list_name = f"{landmark.name}_Point_z"

      data_dict[x_list_name].append(results.pose_landmarks.landmark[landmark].x)
      data_dict[y_list_name].append(results.pose_landmarks.landmark[landmark].y)
      data_dict[z_list_name].append(results.pose_landmarks.landmark[landmark].z)

  if cv2.waitKey(1) & 0xFF == ord('q'):
    break

Processing frame 1
Processing frame 2
Processing frame 3
Processing frame 4
Processing frame 5
Processing frame 6
Processing frame 7
Processing frame 8
Processing frame 9
Processing frame 10
Processing frame 11
Processing frame 12
Processing frame 13
Processing frame 14
Processing frame 15
Processing frame 16
Processing frame 17
Processing frame 18
Processing frame 19
Processing frame 20
Processing frame 21
Processing frame 22
Processing frame 23
Processing frame 24
Processing frame 25
Processing frame 26
Processing frame 27
Processing frame 28
Processing frame 29
Processing frame 30
Processing frame 31
Processing frame 32
Processing frame 33
Processing frame 34
Processing frame 35
Processing frame 36
Processing frame 37
Processing frame 38
Processing frame 39
Processing frame 40
Processing frame 41
Processing frame 42
Processing frame 43
Processing frame 44
Processing frame 45
Processing frame 46
Processing frame 47
Processing frame 48
Processing frame 49
Processing frame 50
Processin

## Save Key points to File

In [79]:
# Creating a DataFrame from the dictionary
df = pd.DataFrame(data_dict)

# Adding a column for the frame number
df.insert(0, 'Frame_Number', frame_numbers)

# Writing the DataFrame to an Excel file
file_name = 'pose_landmark_data_' + video_path.split('/')[-1] + '.xlsx'

df.to_excel(dataset_path + '/' + file_name, index=False)

print(f"Data saved to {file_name}")

Data saved to pose_landmark_data_PushUp.mp4.xlsx


## Reading Key points from File

In [83]:
df_excel = pd.read_excel('/content/DSGP-Group28/Datasets/pose_landmark_data_PushUp.mp4.xlsx')